# For Comprehensions

Indy Scala - November 2016


[Brad Fritz](http://bfritz.com/) | [@bfritz](https://twitter.com/bfritz/) | brad@fewerhassles.com

[List comprehensions](https://en.wikipedia.org/wiki/List_comprehension) (math!) ...

<img alt="https://en.wikipedia.org/wiki/List_comprehension" src="https://wikimedia.org/api/rest_v1/media/math/render/svg/8cc639ecfd1200a7fa8c18ed8624443b6f550ace"/>

Meet for loops ...

```
val list = for {
  x <- 1 to 1000
  if x*x > 3
} yield 2 * x
```










## Examples...

In [ ]:
val grid = for {
    row <- 1 to 2 // generator #1
    col <- 1 to 3 // generator #2
} yield (row, col)

In [ ]:
for (row <- 1 to 2; col <- 1 to 3) println((row, col)) // no `yield`, only side effects

In [ ]:
val squares: Seq[Int] = for {
    x <- 1 to 10
    y <- 1 to 10
    sq = x * y if x == y  // includes definition (`sq = x * y`) and filter (`if x == y`)
} yield sq

In [ ]:
val nestedOptions: Option[String] = for {
    o1 <- Some("string")
    o2 <- None            // will short-circuit here
    o3 <- Some(123)
    _ = println("here!")  // handy for debugging
    o4 <- Some("other string")
} yield (s"$o1 $o2 $o3 $o4")

val output = nestedOptions.getOrElse("one or more values missing")

In [ ]:
case class Error()
case class UserInput()
case class Location()
case class Forecast()

def checkRateLimit(input: UserInput): Either[Error,Unit] = Right(Unit)
def geolocate(input: UserInput): Either[Error,Location] = Right(Location())
def findForecast(loc: Location): Either[Error,Forecast] = Right(Forecast())

val input = UserInput()

val forecast: Either[Error,Forecast] = for {  // `Either` (with right-biased in 2.12) adds lots of power
    rateLimited  <- checkRateLimit(input)
    userLocation <- geolocate(input)
    forecast     <- findForecast(userLocation)
} yield (forecast)

![Erik Bakker Play HttpResult example](Erik_Bakker_play_http_result_example.png)

[https://youtu.be/hGMndafDcc8?t=2130](https://youtu.be/hGMndafDcc8?t=2130)

where:

```
type HttpResult[A] = EitherT[Future, Result, A]
```







## More Formally...

For comprehensions are [syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar) that come in two forms:

```
 1. for ( s ) yield e   // for-comprehension, useful for return value
 2. for ( s ) e         // for-loop, returns `Unit`, useful for side effects
``` 

where **s** is a sequence of

* generators:    `row <- 1 to 2`
* definitions:   `sq = x * y`
* filters:       `if x == y`

and **e** is an expression.

For comprehensions work on paramaterized classes `C[A]` where:

```
def map[B](f: A => B): C[B]
def flatMap[B](f: A => C[B]): C[B]
def filter(p: A => Boolean): C[A]
```

Classes that implement the contract:
* List, Map and many other collections
* Option
* Try
* Either
* Future
* Other monads...but being a monad is not required.

## Quick Review: map, flatMap, filter, foreach

### <a href="http://www.scala-lang.org/api/current/scala/collection/Seq.html#map[B](f:A=>B):Seq[B]">Seq[+A].map()</a>
```
def map[B](f: (A) => B): Seq[B]
```

Example:
```
["a", "b", "c"].map(captialize)  -->  ["A", "B", "C"]
```

### <a href="http://www.scala-lang.org/api/current/scala/collection/Seq.html#flatMap[B](f:A=>scala.collection.GenTraversableOnce[B]):Seq[B]">Seq[+A].flatMap()</a>
```
def flatMap[B](f: (A) => GenTraversableOnce[B]): Seq[B]
```

Example:
```
["abc", "def"].flatMap(singleLetters)  -->  ["a", "b", "c", "d", "e", "f"]
```

### <a href="http://www.scala-lang.org/api/current/scala/collection/Seq.html#filter(p:A=>Boolean):Repr">Seq[+A].filter()</a>
```
def filter(p: (A) => Boolean): Seq[A]
```

Example:
```
["abc", "def"].filter(startsWithA)  -->  ["abc"]
```

### <a href="http://www.scala-lang.org/api/current/scala/collection/Seq.html#foreach(f:A=>Unit):Unit">Seq[+A].foreach()</a>
```
def foreach(f: (A) => Unit): Unit
```

Example:
```
["abc", "def"].foreach(println(_))
abc
def
```

## Desugar

* Simple for-loop

```
 Written:  for (x <- c1; y <- c2; z <- c3) {...}

Desugars:  c1.foreach(x => c2.foreach(y => c3.foreach(z => {...})))
```

* Simple for-comprehension

```
 Written:  for (x <- c1; y <- c2; z <- c3) yield {...}

Desugars:  c1.flatMap(x => c2.flatMap(y => c3.map(z => {...})))
```

* For-comprehension with filter

```
 Written:  for (x <- c; if cond) yield {...}

Desugars:  c.withFilter(x => cond).map(x => {...})
      or:  c.filter(    x => cond).map(x => {...})
```

* For-comprehension with definitions

```
 Written:  for (x <- c; y = ...) yield {...}
 
Desugars:  c.map(x => (x, ...)).map((x, y) => {...})
```

In [ ]:
// When all else fails, let the compiler tell us how it desugars...
reflect.runtime.universe.reify {
    for (r <- 1 to 3; c <- 1 to 10 if c != 5) yield (r, c)
}.tree

// Thanks to Adam Rosien
//   http://arosien.github.io/lovely-for-comps/#slide14
//   https://youtu.be/n_j2hzHQlNI?t=405

Have you seen code like this before?  (Probably in Java.)

![Scala pretending to be Java by Kelsey Gilmore-Innis](Kelsey_Gilmore-Innis_Java-ish_option_checks.png)

[https://youtu.be/MHw-dDxC8Z4?t=980](https://youtu.be/MHw-dDxC8Z4?t=980)

For comprehensions can make it a lot nicer...
![Better Option handling with for comprehension by Kelsey Gilmore-Innis](Kelsey_Gilmore-Innis_for_comprehension.png)

[https://youtu.be/MHw-dDxC8Z4?t=1056](https://youtu.be/MHw-dDxC8Z4?t=1056)

Let's walk through a similar example...

Imagine a Java object graph like:
```
Map<String, Section> sections

Section
  Optional<Sport> sport
  
Sport
  boolean active
  boolean complete
```
...and we want to know if a given section is 1) for a sport, that is 2) active, and 3) complete.

The Java code might look like:
```
public boolean isActiveAndComplete(final Section section) {
    if (section != null) {
        if (section.getSport().isPresent()) {
            Sport sport = section.getSport().get();
            return sport.isActive() && sport.isComplete();
        }
    }
    return false;
}
```

In [ ]:
// Let's see what it looks like in Scala without the for-comprehension sugar...
case class Sport(code: String, active: Boolean, complete: Boolean)
case class Section(sport: Option[Sport])

def isActiveAndComplete(maybeSection: Option[Section]) = {
    maybeSection.flatMap { section =>
        section.sport.map { sport =>
            sport.active && sport.complete
        }
    }.getOrElse(false)
}

// isActiveAndComplete(None)
// isActiveAndComplete(Some(Section(Some(Sport("MBB", true, true)))))

In [ ]:
// ...and with the for-comprehension sugar...
def isActiveAndComplete(maybeSection: Option[Section]) = (
    for {
        section <- maybeSection
        sport <- section.sport
    } yield {
        sport.active && sport.complete
    }
).getOrElse(false)

// isActiveAndComplete(None)
// isActiveAndComplete(Some(Section(Some(Sport("MBB", true, true)))))

## Resources

### Videos
* [Journey to the Heart of the For-Yield](https://youtu.be/MHw-dDxC8Z4)
  <br/>Kelsey Gilmore-Innis at Pacific Northwest Scala 2013
* [Lovely for comprehensions: Scala's most useful syntactic tool](https://youtu.be/n_j2hzHQlNI)
  <br/>Adam Rosien at Scala IO 2014
* [Options in Futures, how to unsuck them](https://youtu.be/hGMndafDcc8)
  <br/>Eric Bakker at ScalaDays 2015 in Amsterdam
  
### Other
 * Scala 2.12 Language Spec [section on for comprehensions](http://www.scala-lang.org/files/archive/spec/2.12/06-expressions.html#for-comprehensions-and-for-loops)
 * Daniel C. Sobral [describes the for-comprehension sugar](http://stackoverflow.com/a/1059501) on StackOverflow
 * [Progamming in Scala](https://www.amazon.com/Programming-Scala-Comprehensive-Step-Step/dp/0981531644) Chapter 23 "For Expressions Revisited"
 * [Scala by Example](http://www.scala-lang.org/sites/default/files/linuxsoft_archives/docu/files/ScalaByExample.pdf) Chapter 10 "For-Comprehensions"